In [1]:
%load_ext autoreload
%autoreload 2
import torch
import torchvision
from torchvision.transforms import transforms
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import confusion_matrix
from data_loader import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import model
torch.set_printoptions(linewidth=120)
from utils import *
from collections import OrderedDict
from run_builder import RunBuilder
from run_manager import RunManager
from torch.utils.tensorboard import SummaryWriter

In [3]:
params = OrderedDict(
    lr = [.01, .001]
    ,batch_size = [1000, 10000]
)

m = RunManager()

for run in RunBuilder.get_runs(params):
    comment = f'-{run}'
    
    data_loader = DataLoader('fashion_mnist',batch_size=run.batch_size)
    network = model.Network()
    optimizer = optim.Adam(
        network.parameters(), lr=run.lr
    )
    
    m.begin_run(run, network, data_loader)

    for epoch in range(3):
        m.begin_epoch()
        
        for batch in data_loader.train_loader:
            images, labels = batch # Get Batch
            preds = network(images) # Pass Batch
            loss = F.cross_entropy(preds, labels) # Calculate Loss
            optimizer.zero_grad() # Zero Gradients
            loss.backward() # Calculate Gradients
            optimizer.step() # Update Weights

            m.track_loss(loss, batch)
            m.track_num_correct(preds, labels)
        m.end_epoch()
    m.end_run()
m.save('results')
        



,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size
0,1,1,1.021181,0.612217,9.069826,9.957067,0.010,1000
1,1,2,0.552542,0.790767,8.452662,18.534813,0.010,1000
2,1,3,0.454399,0.830750,8.717432,27.360851,0.010,1000
3,2,1,2.127951,0.232083,11.246453,16.152480,0.010,10000
4,2,2,1.427631,0.496733,9.200957,25.468428,0.010,10000
5,2,3,0.959315,0.634350,9.341100,34.920359,0.010,10000
6,3,1,1.492499,0.483200,8.232677,8.957187,0.001,1000
7,3,2,0.736502,0.711833,8.648195,17.715897,0.001,1000
8,3,3,0.643043,0.746833,9.499127,27.323145,0.001,1000
9,4,1,2.294487,0.147167,11.099376,16.539533,0.001,10000
